In [ ]:
# FROM PREVIOUS

# Naive exact matching algorithm
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

# Reverse Compliment
def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

# DNA reference genome from FASTA
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

# Read/quality strings from FASTQ
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [ ]:
# 1: First, implement a version of the naive exact matching algorithm that is strand-aware. 
# That is, instead of looking only for occurrences of P in T, additionally look for occurrences of the
# reverse complement of P in T. If P is ACT, your function should find occurrences of both ACT
# and its reverse complement AGT in T.

# If P and its reverse complement are identical (e.g. AACGTT), then a given match offset should be reported only once. 
# So if your new function is called naive_with_rc, then the old naive function and your new naive_with_rc function 
# should return the same results when P equals its reverse complement.

def naive_with_rc(p, t):
    occurences = []
    p_rev = reverseComplement(p)
    # Loop through alignments
    for i in range(len(t) - len(p) + 1):
        # Set marker
        match = True
        # Loop through characters of p
        for j in range(len(p)):
            if not p[j] == t[i+j]:
                match = False
                break
        if match:
            occurences.append(i)
        if not p == p_rev:
            matchRev = True
            for j in range(len(p_rev)):
                if not p_rev[j] == t[i+j]:
                    matchRev = False
                    break
            if matchRev:
                occurences.append(i)
    return occurences

p = 'CCC'
ten_as = 'AAAAAAAAAA'
t = ten_as + 'CCC' + ten_as + 'GGG' + ten_as
occurrences = naive_with_rc(p, t)
print(occurrences)

p = 'CGCG'
t = ten_as + 'CGCG' + ten_as + 'CGCG' + ten_as
occurrences = naive_with_rc(p, t)
print(occurrences)

def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

phix_genome = readGenome('phix.fa')
occurrences = naive_with_rc('ATTA', phix_genome)
print('offset of leftmost occurrence: %d' % min(occurrences))
print('# occurrences: %d' % len(occurrences))


In [ ]:
# 2: Next, download and parse the lambda virus genome, at: 
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

genome = readGenome('lambda_virus.fa')

# Quiz 1. How many times does 𝙰𝙶𝙶𝚃 or its reverse complement (𝙰𝙲𝙲𝚃) occur in the lambda virus genome? 
# E.g. if 𝙰𝙶𝙶𝚃 occurs 10 times and 𝙰𝙲𝙲𝚃 occurs 12 times, you should report 22.
occurences = naive_with_rc('AGGT', genome)
print len(occurences)

# Quiz 2. How many times does 𝚃𝚃𝙰𝙰 or its reverse complement occur in the lambda virus genome?
occurences = naive_with_rc('TTAA', genome)
print len(occurences)

# Quiz 3. What is the offset of the leftmost occurrence of 𝙰𝙲𝚃𝙰𝙰𝙶𝚃 or its reverse complement in the Lambda virus genome? 
# E.g. if the leftmost occurrence of 𝙰𝙲𝚃𝙰𝙰𝙶𝚃 is at offset 40 (0-based) and the leftmost occurrence of its reverse 
# complement 𝙰𝙲𝚃𝚃𝙰𝙶𝚃 is at offset 29, then report 29.
occurences = naive_with_rc('ACTAAGT', genome)
print min(occurences)

# Quiz 4. What is the offset of the leftmost occurrence of 𝙰𝙶𝚃𝙲𝙶𝙰 or its reverse complement in the Lambda virus genome?
occurences = naive_with_rc('AGTCGA', genome)
print min(occurences)

In [ ]:
# 3: As we will discuss, sometimes we would like to find approximate matches for P in T. 
# That is, we want to find occurrences with one or more differences.
# For Questions 5 and 6, make a new version of the 𝚗𝚊𝚒𝚟𝚎 function called 𝚗𝚊𝚒𝚟𝚎_𝟸𝚖𝚖 that allows up to 2 
# mismatches per occurrence. 
# Unlike for the previous questions, do not consider the reverse complement here. 
# We're looking for approximate matches for P itself, not its reverse complement.

def naive_2mm(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        mismatch = 0
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                mismatch += 1
                if mismatch > 2:
                    match = False
                    break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

genome = readGenome('lambda_virus.fa')

# Quiz 5. How many times does 𝚃𝚃𝙲𝙰𝙰𝙶𝙲𝙲 occur in the Lambda virus genome when allowing up to 2 mismatches?
occurences = naive_2mm('TTCAAGCC', genome)
print len(occurences)

# Quiz 6. What is the offset of the leftmost occurrence of 𝙰𝙶𝙶𝙰𝙶𝙶𝚃𝚃 in the Lambda virus genome when allowing up to 2 mismatches?
occurences = naive_2mm('AGGAGGTT', genome)
print min(occurences)

In [ ]:
# Quiz 7. 
# Finally, download and parse the provided FASTQ file containing real DNA sequencing reads derived from a human:
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR037900_1.first1000.fastq

In [44]:
# This dataset has something wrong with it; one of the sequencing cycles is poor quality.

# Report which sequencing cycle has the problem. Remember that a sequencing cycle corresponds to a particular 
# offset in all the reads. For example, if the leftmost read position seems to have a problem consistently across 
# reads, report 0. If the fourth position from the left has the problem, report 3. 
# Do whatever analysis you think is needed to identify the bad cycle. 
# It might help to review the "Analyzing reads by position" video.

def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()
            seq = fh.readline().rstrip()
            fh.readline()
            qual = fh.readline().rstrip()
            if len(seq) == 0:
                break # reached the end
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

# Convert Phred33 to Q
def phred33ToQ(qual):
    return ord(qual) - 33

def qualityByPos(reads):
    sum_quality = [0] * 100 # sum of quality at each position in read
    for read in reads:
        for i in range(len(read)):
            sum_quality[i] += phred33ToQ(read[i])
    return sum_quality.index(min(sum_quality))


_, quals = readFastq('ERR037900_1.first1000.fastq')
qualityByPos(quals)

66